In [158]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from lxml import html
import numpy as np

In [88]:
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
driver.get('https://hoopshype.com/salaries/players/')

In [6]:
players = driver.find_elements_by_xpath('//td[@class="name"]')

In [89]:
players = driver.find_elements_by_xpath('//td[@class="name"]')

players_list = []
for p in range(len(players)):
    players_list.append(players[p].text)

#players_list

In [ ]:
salaries = driver.find_elements_by_xpath('//td[@class="hh-salaries-sorted"]')

salaries_list = []
for s in range(len(salaries)):
    salaries_list.append(salaries[s].text)

## Example 2

In [90]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [91]:
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver = webdriver.Firefox(r'C:\Users\rober\Anaconda3\bin\geckodriver')
driver.get('http://www.platinum.matthey.com/prices/price-charts')

In [42]:
# Wait 20 seconds for page to load
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='chart-content']/div/div[1]/div/div/table/tbody/tr[1]/td[1]")))
except TimeoutException:
    print('Timed out waiting for page to load')
    driver.quit()

Timed out waiting for page to load


In [109]:
# find_elements_by_xpath returns an array of selenium objects.
dates_element = driver.find_elements_by_xpath("""//*[@id="chart-content"]/div/div[1]/div/div/table/tbody/tr""")
#dates_string = dates_element.text
dates_list = []
for p in range(len(dates_element)):
    dates_list.append(dates_element[p].text)

In [110]:
dates_list

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

## lets try the table

In [111]:
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver = webdriver.Firefox(r'C:\Users\rober\Anaconda3\bin\geckodriver')
driver.get('http://www.platinum.matthey.com/prices/price-tables')

In [113]:
# find_elements_by_xpath returns an array of selenium objects.
dates_element = driver.find_elements_by_xpath("""//*[@id="priceTablesFrame"]/table[1]/tbody/tr[8]/th[2]""")
#dates_string = dates_element.text
dates_list = []
for p in range(len(dates_element)):
    dates_list.append(dates_element[p].text)

In [114]:
dates_list

['12nd']

## Metal Prices

In [34]:
option = webdriver.ChromeOptions()
option.add_argument(“ — incognito”)
driver = webdriver.Chrome(r'C:\Users\rober\Anaconda3\bin\chromedriver')
#driver = webdriver.Firefox(r'C:\Users\rober\Anaconda3\bin\geckodriver')
driver.get('http://www.platinum.matthey.com/prices/price-charts')

SyntaxError: invalid character in identifier (<ipython-input-34-0dc4660eb39a>, line 2)

In [32]:
dates = driver.find_elements_by_xpath('//*[@id="chart-content"]/div/div[1]/div/svg/g[2]/g/g/text')

In [33]:
dates = []
for p in range(len(dates)):
    dates_list.append(dates[p].text)

dates

[]

## xpath

In [138]:
request = requests.get('http://www.platinum.matthey.com/prices/price-tables')
data = html.fromstring(request.text)

In [77]:
dates = data.xpath("//*[@id='chart-content']/div/div[2]/text")
print(dates)

[]


In [151]:
request = requests.get('http://www.platinum.matthey.com/prices/price-tables')
data = html.fromstring(request.text)

# Xpath
alldata =[]

for i in data.xpath("//tr[@class]"):
    day = i.xpath('th[2]/text()')
    url = i.xpath('td[4]/text()')
    #year = i.xpath('span[2]/text()')   
    #print(title, url, year)
    alldata.append([day, url])
    
len(alldata)

80

In [209]:
metal_df = pd.DataFrame(alldata)
metal_df.columns = ['day', 'price - USD per troy ounce']
metal_df = metal_df.tail(-2)
metal_df = metal_df.explode('price - USD per troy ounce')
metal_df = metal_df.explode('day')
metal_df['day'] = metal_df['day'].str.replace('\r\n ', '')
metal_df['price - USD per troy ounce'] = metal_df['price - USD per troy ounce'].str.replace('\r\n ', '')
metal_df['day'] = metal_df['day'].str.strip()
metal_df['len'] = metal_df['day'].str.len()
metal_df['count_blank'] = np.where((metal_df['day'].isnull()) & (metal_df['price - USD per troy ounce'].isnull()), 1, 0)
metal_df = metal_df[metal_df.len != 0]
metal_df['flag_delete_row_1'] = np.where(metal_df['len'] > 2, 1, 0)
metal_df['flag_delete_row_2'] = np.where((metal_df['len'].isnull() == True) & (metal_df['count_blank'] == 0), 1, 0)
metal_df = metal_df[metal_df.flag_delete_row_1 != 1]
metal_df = metal_df[metal_df.flag_delete_row_2 != 1]
metal_df['blank_cum_sum'] = metal_df['count_blank'].cumsum()

##dictionary of values I want to map
metal_dict = {0: 'Platinum', 1: 'Palladium', 2: 'Rhodium', 3: 'Iridium', 4: 'Ruthenium'}
metal_df['metal_name'] = metal_df['blank_cum_sum'].map(metal_dict)
metal_df = metal_df[metal_df.count_blank != 1]
metal_df.head(30)

,day,price - USD per troy ounce,len,count_blank,flag_delete_row_1,flag_delete_row_2,blank_cum_sum,metal_name
2,1,1094,1.0,0,0,0,0,Platinum
3,2,1094,1.0,0,0,0,0,Platinum
4,5,1104,1.0,0,0,0,0,Platinum
5,6,1117,1.0,0,0,0,0,Platinum
6,7,1094,1.0,0,0,0,0,Platinum
7,8,1080,1.0,0,0,0,0,Platinum
8,9,1095,1.0,0,0,0,0,Platinum
9,12,1113,2.0,0,0,0,0,Platinum
10,13,1116,2.0,0,0,0,0,Platinum
11,14,1135,2.0,0,0,0,0,Platinum


In [157]:
print(metal_df['date'][0])

['London']


In [122]:
for i in alldata:
    url = (i[0].replace('\r\n        ', ''))
    print(url)

AttributeError: 'list' object has no attribute 'replace'